# Battle of Neighborhoods (Abuja and Lagos)

# Introduction: Business Problem

In this project, we will compare 2 states in Nigeria (Abuja and Lagos) to know where is more suitable for *Business trip and/or Vacation trip*. **Lagos** and **Abuja** are major states in **Nigeria** for business trips or vacations trips.  
We are going to tahe a glimplse on which of the state is more suitable to go on *Business trips or vacation trips*.

## Target Audience
Target audiences for this project does not limit to a person who keeps travelling but everyone. People could simply decide to look for a similar location to do vacation or get the best place in the state to go for vaction or business trip.  
So target for this project is basically everyone who is exploring different places or similar places.

# Data

In order to accomplish this goal, the cities of each state with their latitudes and longitudes are required. The longitude and latitude of each locations in the state were computed manually as well as the . From this data, I need to select the capital cities and using their latitude and longitude values, I need to explore the venues with "art" section around these cities by utilizing Foursquare API.

Then I used the acquired location data to explore the nearby art venues from the Foursquare API. Using the "explore" option, I look for top 25 art venues in 10 km radius for each city center and I get "Venue", "Venue Latitude", "Venue Longitude" and "Venue Category" columns.